In [0]:
df1 = spark.read.parquet("abfss://labdata@dbdeltalabstorageacct.dfs.core.windows.net/invoices/invoices_101_200.parquet")
df2 = spark.read.parquet("abfss://labdata@dbdeltalabstorageacct.dfs.core.windows.net/invoices/invoices_1_100.parquet")
df3 = spark.read.parquet("abfss://labdata@dbdeltalabstorageacct.dfs.core.windows.net/invoices/invoices_201_99457.parquet")

In [0]:
df_union = df1.union(df2).union(df3)
df_union = df_union.select("customer_id", "category", "price", "quantity", "invoice_date")

In [0]:
df_union.write.mode("overwrite").partitionBy("invoice_date").saveAsTable("deltacatalog.deltadb.lc_ex1")

In [0]:
%sql
OPTIMIZE deltacatalog.deltadb.lc_ex1
ZORDER BY customer_id; 

In [0]:
%sql
SELECT count(*)
FROM FROM deltacatalog.deltadb.lc_ex1; 

FROM
99457


In [0]:
df_union.write.mode("overwrite").clusterBy("invoice_date", "customer_id").saveAsTable("deltacatalog.deltadb.lc_ex2")

In [0]:
%sql
SELECT count(*)
FROM FROM deltacatalog.deltadb.lc_ex2; 

FROM
99457


In [0]:
%%time
spark.sql(
    """
    SELECT category,
        SUM(price * quantity) AS total_sales
    from deltacatalog.deltadb.lc_ex1
    WHERE (invoice_date BETWEEN '2021-01-01' AND '2023-12-31') AND 
        customer_id = 201
    GROUP BY category
    """
)

CPU times: user 2.03 ms, sys: 268 µs, total: 2.3 ms
Wall time: 145 ms


DataFrame[category: string, total_sales: double]

In [0]:
%%time
spark.sql(
    """
    SELECT category,
        SUM(price * quantity) AS total_sales
    from deltacatalog.deltadb.lc_ex2
    WHERE (invoice_date BETWEEN '2021-01-01' AND '2023-12-31') AND 
        customer_id = 201
    GROUP BY category
    """
)

CPU times: user 3.19 ms, sys: 509 µs, total: 3.7 ms
Wall time: 152 ms


DataFrame[category: string, total_sales: double]